In [24]:
# import fasttext
# import pandas as pd
import re

In [25]:
# # model = fasttext.train_supervised(input=TRAIN_FILEPATH, lr=1.0, epoch=100,
# #                              wordNgrams=2, bucket=200000, dim=300, loss='hs',
# #                              pretrainedVectors=VECTORS_FILEPATH)
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r",", " ", string) 
    string = re.sub(r"!", " ", string) 
    string = re.sub(r"\(", " ", string) 
    string = re.sub(r"\)", " ", string) 
    string = re.sub(r"\?", " ", string) 
    string = re.sub(r"\s{2,}", " ", string)   
    string = re.sub(" \d+", " ", string)
    return  string.strip().lower()

In [26]:
import pandas as pd
TRAIN_FILEPATH = "../Translated/cleaned/train.csv"
TEST_FILEPATH = "../Translated/cleaned/test.csv"
SS_FILEPATH = "../data/SampleSubmission.csv"
VECTORS_FILEPATH = ""
train = pd.read_csv(TRAIN_FILEPATH)
test = pd.read_csv(TEST_FILEPATH)
ss = pd.read_csv(SS_FILEPATH)

In [27]:
# # train.Text.str.lower().str.split()
# # from collections import Counter
# # result = Counter(" ".join(train['cleaned'].values.tolist()).split(" ")).items()
# # result
train["Text"] =train.Text.apply(lambda x: clean_str(x))
test["Text"] =test.Text.apply(lambda x: clean_str(x))


In [28]:
# len(result)
# test
# result
# corpus = 
# train.Text.values.dtype
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(corpus)
def keep_only(x,string):
    return ' '.join(string.split()[:x])
    

In [29]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.3.0


In [30]:
data = train.copy()
train_size = int(len(data) * .9)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

Train size: 1292
Test size: 144


In [31]:
train_cat, test_cat = train_test_split(data['Label'], train_size)
train_text, test_text = train_test_split(data['Text'], train_size)

In [41]:
max_words = 44412
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)
tokenize.fit_on_texts(pd.concat([data['Text'],test['Text'] ], ignore_index = True)) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [60]:
# tesst = tokenize.texts_to_matrix(test['Text'])
data_train = tokenize.texts_to_matrix(pd.concat([data['Text'],test['Text'] ], ignore_index = True))
data_train.shape

(2056, 44412)

In [43]:
# test.shape
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [44]:
# from sklearn.preprocessing import LabelBinarizer, StandardScaler
# lb = LabelBinarizer()
# y_train_b = lb.fit_transform(train['Label'])

num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [62]:
from keras import backend as K 

# Do some code, e.g. train and save model

K.clear_session()
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

encoder = models.Sequential()
encoder.add(layers.Dense(4048, input_shape=(max_words,)))
# model.add(layers.BatchNormalization())
encoder.add(layers.Activation('selu'))
# model.add(layers.Dropout(drop_ratio))
encoder.add(layers.Dense(2048))
# model.add(layers.BatchNormalization())
encoder.add(layers.Activation('selu'))
encoder.add(layers.Dense(512))
# model.add(layers.BatchNormalization())
encoder.add(layers.Activation('selu'))
encoder.add(layers.Dense(128))
# model.add(layers.BatchNormalization())
encoder.add(layers.Activation('selu'))

decoder = models.Sequential()
decoder.add(layers.Dense(512, input_shape=(128,)))
# model.add(layers.BatchNormalization())
decoder.add(layers.Activation('selu'))
decoder.add(layers.Dense(2048))
# model.add(layers.BatchNormalization())
decoder.add(layers.Activation('selu'))

decoder.add(layers.Dense(4048))
# model.add(layers.BatchNormalization())
decoder.add(layers.Activation('selu'))

decoder.add(layers.Dense(44412))
# model.add(layers.BatchNormalization())
# encoder.add(layers.Activation('selu'))

# model.add(layers.Dense(512))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))
# model.add(layers.Dense(128))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))

# model.add(layers.Dropout(drop_ratio))
# model.add(layers.Dense(num_classes))
decoder.add(layers.Activation('softmax'))

model = models.Sequential([encoder, decoder])
model.compile(loss='binary_crossentropy',
              optimizer='adam')

In [63]:
# features = X_train.columns[:-2]
# ((x_train.sum(axis = 0)) == 0).sum()
batch_size = 32
epochs = 3
history = model.fit(data_train, data_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
 1/58 [..............................] - ETA: 9s - loss: 0.0382

ResourceExhaustedError:  OOM when allocating tensor with shape[44412,4048] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node gradient_tape/sequential_2/sequential/dense/MatMul (defined at <ipython-input-63-ebd0d3e2de8a>:9) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_8987]

Function call stack:
train_function


In [48]:
x_train.shape

(1292, 44412)

In [114]:
len(features)

1924

In [54]:
# tes = pd.DataFrame(mat[:train.shape[0]])

In [122]:
# train.Text[0]
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import nltk
nltk.download('stopwords')
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs
from sklearn.preprocessing import LabelBinarizer
sns.set_style("whitegrid")
np.random.seed(0)

DATA_PATH = '../data'
EMBEDDING_DIR = '../Vectors'

MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amakr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
# print('loading word embeddings...')
# embeddings_index = {}
# f = codecs.open('../Vectors/wiki.ny.vec', encoding='utf-8')
# for line in tqdm(f):
#     values = line.rstrip().rsplit(' ')
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# print('found %s word vectors' % len(embeddings_index))

In [116]:
# model = fasttext.load_model()
# train[train.Label == "POLITICS"].Text[0]

In [117]:
# dir(model)
# print("num train: ", train.shape[0])
# print("num test: ", test.shape[0])

# label_names = train.Label.unique().tolist()
# y_train = train.Label.values

# # #visualize word distribution
# train['doc_len'] = train['Text'].apply(lambda words: len(words.split(" ")))
# max_seq_len = np.round(train['doc_len'].mean() + train['doc_len'].std()).astype(int)
# sns.distplot(train['doc_len'], hist=True, kde=True, color='b', label='doc len')
# plt.axvline(x=max_seq_len, color='k', linestyle='--', label='max len')
# plt.title('article length'); plt.legend()
# plt.show()

In [118]:
# # # model.predict(train.cleaned[0])
# # model = fasttext.train_supervised(input=TRAIN_FILEPATH, lr=1.0, epoch=100,
# #                              wordNgrams=2, bucket=200000, dim=300, loss='hs')
# # #                              pretrainedVectors="../Vectors/wiki.ny.vec")
# raw_docs_train = train['Text'].tolist()
# raw_docs_test = test['Text'].tolist() 
# num_classes = len(label_names)

# print("pre-processing train data...")
# processed_docs_train = []
# for doc in tqdm(raw_docs_train):
#     tokens = tokenizer.tokenize(doc)
#     filtered = [word for word in tokens if word not in stop_words]
#     processed_docs_train.append(" ".join(filtered))
# #end for

# processed_docs_test = []
# for doc in tqdm(raw_docs_test):
#     tokens = tokenizer.tokenize(doc)
#     filtered = [word for word in tokens if word not in stop_words]
#     processed_docs_test.append(" ".join(filtered))
# #end for

# print("tokenizing input data...")
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS+1, lower=True, char_level=False)
# tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
# word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
# word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
# word_index = tokenizer.word_index
# print("dictionary size: ", len(word_index))

# #pad sequences
# word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
# word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [119]:
# train.Text
# i
# word_index

In [238]:
# word_seq_train.shape
#training params
batch_size = 64 
num_epochs =  50

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-3

In [239]:
# print('preparing embedding matrix...')
# words_not_found = []
# nb_words = min(MAX_NB_WORDS, len(word_index))
# embedding_matrix = np.zeros((nb_words, embed_dim))
# for word, i in word_index.items():
#     if i >= nb_words:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if (embedding_vector is not None) and len(embedding_vector) > 0:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
#     else:
#         words_not_found.append(word)
# print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [138]:
# print("sample words not found: ", np.random.choice(words_not_found, 100))
# len(words_not_found)
# lb = LabelBinarizer()
# y_train_b = lb.fit_transform(y_train) 
X_train[features][56]

0      -0.247346
1      -0.816465
2      -0.603540
3       0.370478
4      -0.201107
          ...   
1431   -0.830241
1432   -4.398766
1433    7.091032
1434    1.153235
1435   -4.859901
Name: 56, Length: 1436, dtype: float64

In [131]:
# y_train_b[0]
num_classes = y_train_b.shape[1]

In [202]:
X_train[features].max()

0        81.021802
1        89.284926
2       143.630518
3        11.769739
4       375.905040
           ...    
1919     46.571552
1920     51.781500
1921     58.180361
1922     60.925842
1923     38.065951
Length: 1924, dtype: float64

In [246]:
print("training CNN ...")
model = Sequential()
model.add(keras.Input(shape=(X_train.shape[1]-2,1)))
# model.add(Embedding(63440, embed_dim, trainable=False))
# Conv1D(filters=1, kernel_size=10 ,strides=10,     
#                   ,kernel_initializer= 'uniform',      
#                   activation= 'relu')
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

training CNN ...
Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 1924, 64)          512       
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 962, 64)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 962, 64)           28736     
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, 481, 64)           0         
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 481, 64)           28736     
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 240, 64)           0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 

In [247]:
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=6, verbose=1)
callbacks_list = [early_stopping]
# y_train = np.asarray(y_train).reshape((-1,1))


In [248]:
hist = model.fit(X_train[features].values.reshape(X_train[features].shape[0],X_train[features].shape[1],1 ), y_train_b, batch_size=batch_size, epochs=num_epochs,shuffle = True, callbacks=callbacks_list, validation_split=0.1, verbose=2)

Epoch 1/50
21/21 - 7s - loss: 0.3716 - accuracy: 0.0921 - val_loss: 0.2726 - val_accuracy: 0.0972
Epoch 2/50
21/21 - 7s - loss: 0.2809 - accuracy: 0.1385 - val_loss: 0.2608 - val_accuracy: 0.2014
Epoch 3/50
21/21 - 7s - loss: 0.2666 - accuracy: 0.1486 - val_loss: 0.2526 - val_accuracy: 0.1944
Epoch 4/50
21/21 - 7s - loss: 0.2535 - accuracy: 0.1827 - val_loss: 0.2453 - val_accuracy: 0.2014
Epoch 5/50
21/21 - 7s - loss: 0.2457 - accuracy: 0.1587 - val_loss: 0.2369 - val_accuracy: 0.2014
Epoch 6/50
21/21 - 7s - loss: 0.2349 - accuracy: 0.1950 - val_loss: 0.2323 - val_accuracy: 0.1944
Epoch 7/50
21/21 - 7s - loss: 0.2254 - accuracy: 0.1803 - val_loss: 0.2194 - val_accuracy: 0.2014
Epoch 8/50
21/21 - 7s - loss: 0.2131 - accuracy: 0.1749 - val_loss: 0.2080 - val_accuracy: 0.2014
Epoch 00008: early stopping


In [ ]:
# len(train.Label.unique().tolist())
# word_seq_train.shape
# labels
a = model.predict(word_seq_test)
# word_seq_train.max()
# word_seq_test.shape

In [ ]:
# word_seq_train.shape
# a[0]
word_seq_test.max()

In [ ]:
# logits
y_train_b[0]*a[0]

In [ ]:
word_seq_train.shape

In [ ]:
y_train.shape

In [113]:
from googletrans import Translator


In [114]:
translator = Translator()
# train.Text[0]

In [116]:
# translator.translate('안녕하세요.')

In [118]:
# cleaned_train = pd.read_csv('../Translated/cleaned/train.csv')
# test.char_len.max()
# train.Text.to_csv('../Translated/train1.csv',index = False)

In [120]:
# cleaned_train.Text.to_csv('../Translated/train1.csv',index = False)
# plt.figure()
# plt.hist(test['char_len'])
# plt.show()

In [59]:
# 1436*2
train['char_len'] = train['Text'].apply(lambda x: len(str(x)))
test['char_len'] = test['Text'].apply(lambda x: len(str(x)))


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(max_features=5000, stop_words='english')

In [63]:
X_dtm = vect.fit_transform(train.Text)


In [65]:
# X_dtm
test_X_dtm = vect.transform(test.Text)


In [66]:
from keras.preprocessing import text

In [67]:
tokenizer = text.Tokenizer()

In [68]:
tokenizer.fit_on_texts(train.Text)

In [74]:
a = tokenizer.texts_to_sequences(train.Text)

In [79]:
max(max(a))

49694

In [78]:
a = tokenizer.texts_to_sequences(test.Text)

In [82]:
len(set(a[0]))

170

In [222]:
A = X_train[features]

In [225]:
X_train[features].reshape(-1,X_train[features].shape[0],X_train[features].shape[1] )

AttributeError: 'DataFrame' object has no attribute 'reshape'